<a href="https://colab.research.google.com/github/mralamdari/CV-Object-Detection-Projects/blob/main/Flowers_Recognition's_optimal_approach(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hello 👋
####This repository will give you a simple approach (but an effective one) to code in PyTorch that can be used recursively for any other problems in Machine Learning/Deep Learning Field.


#####Unfortunately, the Ram crushes in the Colab, so I can't train it with PyTorch; if you have more ram, feel free to run this code, but if you don't have a powerful system to run it, you can run this code in [My Kaggle Notebook](https://www.kaggle.com/code/mralamdari/flowers-recognition-s-optimal-approach-PyTorch), there are more models with trainning results in this notebook and you can easily edit and run it.
#####You can get more details on this project and learn about object recognition on my article on medium; [How to do Object Recognition with PyTorch(Keras) the Easiest way](https://medium.com/@mr.alamdari/imagehow-to-do-object-recognition-with-PyTorch-keras-the-easiest-way-23c7ab9604c7)

# 1.Import Essential Libraris


In [ ]:
import os
import cv2
import tqdm
import torch
import warnings
import matplotlib
import torchvision
import numpy as np
import pandas as pd
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics, preprocessing

In [ ]:
warnings.filterwarnings('ignore')
warnings.filterwarnings('always')

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'
!kaggle datasets download -d alxmamaev/flowers-recognition
!unzip \*.zip && rm *.zip

#2. Data
The Dataset in this project contains 4242 images of flowers; the data collection is based on the data Flickr, Google Images, and Yandex images, and it is used to recognize plants from the photo. There are five kinds of flowers: daisy, dandelion, rose, sunflower, and tulip, and each class has about 800 pictures of different sizes but not high resolutions. You can access the dataset here.

In PyTorch, you can readily do DataAugmentation and grow your dataset's size; since Neural Networks need more data to train, it will enhance the model's performance.

### Transformer

In [ ]:
path_folder = '/content/flowers'

In [ ]:
mean = (0.4124234616756439, 0.3674212694168091, 0.2578217089176178)
std = (0.3268945515155792, 0.29282665252685547, 0.29053378105163574)

In [ ]:
transformer = {
    'original': torchvision.transforms.Compose([
                                               torchvision.transforms.Resize((115, 115)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)
    ]),
    'dataset1': torchvision.transforms.Compose([
                                               torchvision.transforms.Resize((115, 115)),
                                               torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                                               torchvision.transforms.RandomRotation(5),
                                               torchvision.transforms.RandomAffine(degrees=11, translate=(0.1, 0.1), scale=(0.8, 0.8)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)
    ]),
    'dataset2': torchvision.transforms.Compose([
                                               torchvision.transforms.Resize((115, 115)),
                                               torchvision.transforms.RandomHorizontalFlip(),
                                               torchvision.transforms.RandomRotation(10),
                                               torchvision.transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.RandomErasing(inplace=True, scale=(0.01,  0.23)),
                                               torchvision.transforms.Normalize(mean, std)
    ]),
    'dataset3': torchvision.transforms.Compose([
                                               torchvision.transforms.Resize((115, 115)),
                                               torchvision.transforms.RandomHorizontalFlip(p=0.5),
                                               torchvision.transforms.RandomRotation(15),
                                               torchvision.transforms.RandomAffine(degrees=11, translate=(0.1, 0.1), scale=(0.8, 0.8)),
                                               torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean, std)
    ]),
}

### Train/Test/Val Split

In [ ]:
#all dataset ==> train&val + test
original = torchvision.datasets.ImageFolder(path, transform=transformer['original'])
train_val, test = model_selection.train_test_split(original, test_size=0.2, random_state=32, shuffle=True)


In [ ]:
# train_val  ==> train + val + dataset1 + dataset2 + dataset3
train_val = torch.utils.data.ConcatDataset([train_val,
                                torchvision.datasets.ImageFolder(path, transform=transformer['dataset1']),
                                torchvision.datasets.ImageFolder(path, transform=transformer['dataset2']),
                                torchvision.datasets.ImageFolder(path, transform=transformer['dataset3'])])

In [ ]:
train, val = model_selection.train_test_split(train_val, test_size=0.1, random_state=32, shuffle=True)

### Data Loader

In [ ]:
batch_size=32
data_loaders = {
    'train': torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=2, pin_memory=True),
    'val': torch.utils.data.DataLoader(val, batch_size=batch_size, num_workers=2, pin_memory=True),
    'test': torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2, pin_memory=True)
}

dataset_sizes = {
    'train': len(train),
    'val': len(val),
    'test': len(test)
}

In [ ]:
dataset_sizes

#### How ImBalance is ourdaset

In [ ]:
dic = {}
for cls in original.classes:
  dic[cls] = len(os.listdir(f'{path}/{cls}'))

samplesize = pd.DataFrame(dic, index=[0])
samplesize

In [ ]:
sns.barplot(data=samplesize)

# 3.Visualization

In [ ]:
z, _ = next(iter(data_loaders['test']))
print(z.mean(), z.std())
img_norm = z[0].permute(1, 2, 0).numpy()
plt.imshow(img_norm)

In [ ]:
z, _ = next(iter(data_loaders['val']))
print(z.mean(), z.std())
img_norm = z[0].permute(1, 2, 0).numpy()
plt.imshow(img_norm)

In [ ]:
def plot_imgs(imgs, nrows=5, ncols=5):
  fig, ax = plt.subplots(nrows, ncols, figsize=(nrows*5, ncols*3))
  index = 0
  for row in range(nrows):
    for col in range(ncols):
      img = matplotlib.image.imread(imgs[index][0])
      ax[row][col].imshow(img)
      ax[row][col].axis('off')
      ax[row][col].set_title(imgs[index][1], fontsize=15)
      index += 1

In [ ]:
def rand_imgs(original, img_folder=path, count=25):
  rand_imgs = []
  categories = original.classes
  for cat in categories:
    folder_path = f"{img_folder}/{cat}"
    imgs_list = os.listdir(folder_path)
    selected_imgs = np.random.choice(imgs_list, count//len(categories))
    rand_imgs.extend([(f'{folder_path}/{img_path}', cat) for img_path in selected_imgs])
  np.random.shuffle(rand_imgs)
  return rand_imgs

In [ ]:
my_imgs = rand_imgs(original, path, 15)

In [ ]:
plot_imgs(my_imgs, 5, 3)

In [ ]:
def plot_batch(data_loader):
  for imgs, labels in data_loader:
    fig, ax = plt.subplots(figsize=(25, 25))
    ax.imshow(torchvision.utils.make_grid(imgs[:60], nrow=10).permute(1, 2, 0))
    ax.set_title('Augmented Images')
    break

In [ ]:
plot_batch(data_loaders['train'])

In [ ]:
plot_batch(data_loaders['val'])

In [ ]:
plot_batch(data_loaders['test'])

# 4.Train

In [ ]:
def accuracy(outputs, labels):
  _, preds = torch.max(outputs, dim=1)
  return torch.tensor(torch.sum(preds==labels).item()/len(preds)), preds

In [ ]:
path2weights = '/content/models/'
os.makedirs(path2weights, exist_ok=True)

In [ ]:
def get_lr(opt):
  for param_group in opt.param_groups:
    return param_group['lr']

In [ ]:
def loss_epoch(model, data_loader, dataset_sizes, criterion, optimizer, scheduler, sanity_check, phase):
  running_loss = 0.0
  running_corrects = 0.0

  for input, labels in data_loader:
    inputs = input.to(device)
    labels = labels.to(device)
  
  with torch.set_grad_enabled(phase=='train'):
    output = model(inputs)
    loss = criterion(output, labels)
    _, pred = torch.max(output, 1)
    # pred = output.argmax(dim=1, keepdim=True)

  if phase == 'train':
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # current_lr = get_lr(optimizer)
    # print(optimizer)
    # scheduler.step(loss)

  # running_corrects += pred.eq(labels.view_as(pred)).sum().item()
  running_corrects += torch.sum(pred == labels.data)
  
  if device == 'cpu':
    running_loss += loss.item() * inputs.size(0)
    # running_loss += loss.item()
  else:
    running_loss += loss.cpu().detach().numpy()
  
##############################?????????????????????????????????
  # if phase == 'train':
  #   acc = 100 * running_corrects.double() / dataset_sizes
  #   scheduler.step(acc)
  
  epoch_loss = running_loss / dataset_sizes
  epoch_acc = running_corrects.double() / dataset_sizes
  # return epoch_loss, epoch_acc, current_lr
  return epoch_loss, epoch_acc

In [ ]:
def train_val(model, params, requires_grad_param=False, trainable_layers=0):
  
  model_name = params['model_name']
  num_epochs = params['epochs']
  optimizer = params['optimizer']
  criterion = params['criterion']
  scheduler = params['scheduler'][0]
  data_loaders = params['data_loaders']
  dataset_sizes = params['dataset_sizes']
  path2weights = params['path2weights']
  sanity_check = params['sanity_check']

  epoch_lr = 0
  #A dictionary to save Loss's history and accuracy's history
  loss_history = {'train': [], 'val':[]}
  accuracy_history = {'train': [], 'val':[]}
  lr = []

  model_params_len = len(list(model.parameters()))
  for i, param in enumerate(model.parameters()):
    if model_params_len - i > trainable_layers:
      param.requires_grad == requires_grad_param

  model.to(device) 
  best_accuracy = 0.0
  best_loss = float('inf')
  best_model = copy.deepcopy(model.state_dict())

  for epoch in range(num_epochs):
    
    for phase in ['train', 'val']:
      start = time.time()  
      if phase == 'train':
        model.train()
      else:
        model.eval()
  
      # epoch_loss, epoch_acc, epoch_lr = loss_epoch(model, data_loaders[phase], dataset_sizes[phase], criterion, optimizer, scheduler, sanity_check, phase)
      epoch_loss, epoch_acc = loss_epoch(model, data_loaders[phase], dataset_sizes[phase], criterion, optimizer, scheduler, sanity_check, phase)
      # epoch_lr = get_lr(optimizer)
      loss_history[phase].append(epoch_loss)
      accuracy_history[phase].append(epoch_acc)
      # lr.append(epoch_lr)

      print(f'{phase.upper()} ==> Epoch: {epoch+1}/{num_epochs} - Loss: {epoch_loss}, Accuracy: {epoch_acc}, lr: {epoch_lr}')
    
    print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {loss_history['train'][-1]:.4f}, Train Accuracy: %{accuracy_history['train'][-1]*100:.3f}, Val Loss: {loss_history['val'][-1]:.4f}, Val Accuracy: %{accuracy_history['val'][-1]*100:.3f}")


    # if phase == 'val':
    #   during = time.time() - start
    #   print(f'Time: {during//60}m {during%60}s')
    #   print('======'*5)
    if phase == 'val' and epoch_loss < best_loss:
      best_loss = epoch_loss
      best_model = copy.deepcopy(model.state_dict())
      torch.save(best_model, f'{path2weights}{model_name}.h5')
      print(f'The best Model has been saved with loss: {best_loss}!!!')

############################# HOOOOOOOOOOOOOOOOOOOOOOOOOw To Correct it?
    # scheduler.step(optimizer)
    # if lr[-1] != get_lr(optimizer):
    #   print('loading best model weights')
    #   model.load_state_dict(best_model)
    # print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {loss_history['train'][-1]:.4f}, Train Accuracy: %{accuracy_history['train'][-1]*100:.3f}, Val Loss: {loss_history['val'][-1]:.4f}, Val Accuracy: %{accuracy_history['val'][-1]*100:.3f}")
    

  
  during = time.time() - start
  print(f'{phase.upper()} ===> Time: {during//60}m {during%60}s')
  print('======'*5)

  model.load_state_dict(best_model)        
  return model, loss_history, accuracy_history

# 5.Models

## VGG16

In [ ]:
vgg16 = torchvision.models.vgg16(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False

vgg16.classifier = torch.nn.Linear(in_features=vgg16.classifier[6].in_features,
                                  out_features=len(original.classes), 
                                  bias=True)

In [ ]:
optmizer_vgg16 = torch.optim.Adam(vgg16.classifier.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
scheduler_vgg16 = torch.optim.lr_scheduler.ReduceLROnPlateau(optmizer_vgg16, mode='max', patience=3, verbose=1)
epochs = 10
batch_size = 32
params_vgg16 = {
    'epochs': epochs,
    'model_name': 'vgg16',
    'data_loaders': data_loaders,
    'criterion': torch.nn.CrossEntropyLoss(),
    'optimizer': optimizer_vgg16,
    'dataset_sizes': dataset_sizes,
    'scheduler': scheduler_vgg16,
    'path2weights': path2weights,
    'sanity_check': True
}

## VGG19

In [ ]:
vgg19 = torchvision.models.vgg19(pretrained=True)

for param in vgg19.parameters():
    param.requires_grad = False

vgg19.classifier = torch.nn.Linear(in_features=vgg19.classifier[6].in_features,
                                  out_features=len(original.classes), 
                                  bias=True)

In [ ]:
optmizer_vgg19 = torch.optim.Adam(vgg19.classifier.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
scheduler_vgg19 = torch.optim.lr_scheduler.ReduceLROnPlateau(optmizer_vgg19, mode='max', patience=3, verbose=1)
epochs = 10
batch_size = 32
params_vgg19 = {
    'epochs': epochs,
    'model_name': 'vgg19',
    'data_loaders': data_loaders,
    'criterion': torch.nn.CrossEntropyLoss(),
    'optimizer': optimizer_vgg19,
    'dataset_sizes': dataset_sizes,
    'scheduler': scheduler_vgg19,
    'path2weights': path2weights,
    'sanity_check': True
}

## Inceptionv3

In [ ]:
inceptionv3 = torchvision.models.inception_v3(pretrained=True)

for param in inceptionv3.parameters():
    param.requires_grad = False

inceptionv3.classifier = torch.nn.Linear(in_features=inceptionv3.fc.in_features,
                                  out_features=len(original.classes), 
                                  bias=True)

In [ ]:
optmizer_inceptionv3 = torch.optim.Adam(inceptionv3.fc.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
scheduler_inceptionv3 = torch.optim.lr_scheduler.ReduceLROnPlateau(optmizer_inceptionv3, mode='max', patience=3, verbose=1)
epochs = 10
batch_size = 32
params = {
    'epochs': epochs,
    'model_name': 'inceptionv3',
    'data_loaders': data_loaders,
    'criterion': torch.nn.CrossEntropyLoss(),
    'optimizer': optimizer_inceptionv3,
    'dataset_sizes': dataset_sizes,
    'scheduler': scheduler_inceptionv3,
    'path2weights': path2weights,
    'sanity_check': True
}

## ResNet50

In [ ]:
resnet50 = torchvision.models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = False

resnet50.classifier = torch.nn.Linear(in_features=resnet50.fc.in_features,
                                  out_features=len(original.classes), 
                                  bias=True)

In [ ]:
optimizer_resnet50 = torch.optim.Adam(resnet50.fc.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
scheduler_resnet50 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_resnet50, mode='max', patience=3, verbose=1)
epochs = 10
batch_size = 32
params = {
    'epochs': epochs,
    'model_name': 'resnet50',
    'data_loaders': data_loaders,
    'criterion': torch.nn.CrossEntropyLoss(),
    'optimizer': optimizer_resnet50,
    'dataset_sizes': dataset_sizes,
    'scheduler': scheduler_resnet50,
    'path2weights': path2weights,
    'sanity_check': True
}

## EfficientNet B2

In [ ]:
efficientnetb2 = torchvision.models.efficientnet_b2(pretrained=True)

for param in efficientnetb2.parameters():
    param.requires_grad = False

efficientnetb2.classifier = torch.nn.Linear(in_features=efficientnetb2.classifier[1].in_features,
                                  out_features=len(original.classes), 
                                  bias=True)

In [ ]:
optimizer_efficientnetb2 = torch.optim.Adam(efficientnetb2.classifier.parameters(), lr=1e-4, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
scheduler_efficientnetb2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_efficientnetb2, mode='max', patience=3, verbose=1)
epochs = 10
batch_size = 32
params = {
    'epochs': epochs,
    'model_name': 'efficientnetb2',
    'data_loaders': data_loaders,
    'criterion': torch.nn.CrossEntropyLoss(),
    'optimizer': optimizer_efficientnetb2,
    'dataset_sizes': dataset_sizes,
    'scheduler': scheduler_efficientnetb2,
    'path2weights': path2weights,
    'sanity_check': True
}